#### Background 
Our users have points in their accounts. Users only see a single balance in their accounts. But for reporting purposes we actually track their points per payer/partner. In our system, each transaction record contains: payer (string), points (integer), timestamp (date). 

For earning points it is easy to assign a payer, we know which actions earned the points. And thus which partner should be paying for the points. 

When a user spends points, they don't know or care which payer the points come from. But, our accounting team does care how the points are spent. There are two rules for determining what points to "spend" first:

* We want the oldest points to be spent first (oldest based on transaction timestamp, not the order they’re received) 
* We want no payer's points to go negative. 

#### We expect your web service to 

Provide routes that: 
* Add transactions for a specific payer and date. 
* Spend points using the rules above and return a list of { "payer": <string>, "points": <integer> } for each call. 
* Return all payer point balances. 

Note: 
* We are not defining specific requests/responses. Defining these is part of the exercise. 
* We don’t expect you to use any durable data store. Storing transactions in memory is acceptable for the exercise.

#### Example
suppose you call your add transaction route with the following sequence of calls:

In [1]:
transaction_list = [{"payer": "DANNON", "points": 1000, "timestamp": "2020-11-02T14:00:00Z" }
,{"payer": "UNILEVER", "points": 200, "timestamp": "2020-10-31T11:00:00Z" }
,{"payer": "DANNON", "points": -200, "timestamp": "2020-10-31T15:00:00Z" }
,{"payer": "MILLER COORS","points": 10000,"timestamp": "2020-11-01T14:00:00Z" }
,{"payer": "DANNON", "points": 300, "timestamp": "2020-10-31T10:00:00Z" }]

#Then you call your spend points route with the following request: 

spent_points = { "points": 5000 }

# The expected response form the spend call would be:

spent_points_reponse = [{ "payer": "DANNON", "points": -100 }
,{ "payer": "UNILEVER", "points": -200 }
,{ "payer": "MILLER COORS", "points": -4700 }]

# A subsequent call to the points balance route, after the spend, should returns the following results:

point_balance = { "DANNON": 1000, "UNILEVER": 0, "MILLE COORS": 5300 }


In [2]:

# Created a method to sort the transaction_list:
sorted_transaction_list = sorted(transaction_list, key=lambda d: d['timestamp'])
sorted_transaction_list

[{'payer': 'DANNON', 'points': 300, 'timestamp': '2020-10-31T10:00:00Z'},
 {'payer': 'UNILEVER', 'points': 200, 'timestamp': '2020-10-31T11:00:00Z'},
 {'payer': 'DANNON', 'points': -200, 'timestamp': '2020-10-31T15:00:00Z'},
 {'payer': 'MILLER COORS',
  'points': 10000,
  'timestamp': '2020-11-01T14:00:00Z'},
 {'payer': 'DANNON', 'points': 1000, 'timestamp': '2020-11-02T14:00:00Z'}]

In [3]:

points2subtract = spent_points['points']
position_transaction_list = 0

payer_list = list()

count_switch = True

while position_transaction_list < len(transaction_list):

    dictionary_element = sorted_transaction_list[position_transaction_list]
    
    points = dictionary_element['points']
    if count_switch == True:
        if points <= points2subtract:
            points2subtract = points2subtract - points
            dictionary_element['points'] = 0
        else:
            points = points - points2subtract
            dictionary_element['points'] = points
            count_switch = False
    
    position_transaction_list = position_transaction_list + 1

    if dictionary_element['payer'] not in payer_list:
        payer_list.append(dictionary_element['payer'])
sorted_transaction_list

[{'payer': 'DANNON', 'points': 0, 'timestamp': '2020-10-31T10:00:00Z'},
 {'payer': 'UNILEVER', 'points': 0, 'timestamp': '2020-10-31T11:00:00Z'},
 {'payer': 'DANNON', 'points': 0, 'timestamp': '2020-10-31T15:00:00Z'},
 {'payer': 'MILLER COORS',
  'points': 5300,
  'timestamp': '2020-11-01T14:00:00Z'},
 {'payer': 'DANNON', 'points': 1000, 'timestamp': '2020-11-02T14:00:00Z'}]

In [4]:
payer_list 

['DANNON', 'UNILEVER', 'MILLER COORS']

In [5]:
# sorted_transaction_list[''] 


report = {}

for payer in payer_list:
    x = [element for element in sorted_transaction_list if element['payer'] == payer]
    report[payer] = sum(item['points'] for item in x)

report

{'DANNON': 1000, 'UNILEVER': 0, 'MILLER COORS': 5300}

In [48]:

def transaction_return(transaction_list: list , spent_points: int, point_balance=False):
    # transaction_list should be a list of dictionaries with the follwing schema
    # {'payer': str, 'points': int, 'timestamp': str}
    
    # define variables:
    position_transaction_list = 0
    payer_list = list()
    cost2payer_list = list()
    count_switch = True
    balance_report = {}
    cost_report = {}


    # Get the value of the points spent
    points2subtract = spent_points['points']


    # Sort the list of dictionaries base on the timestamp. 
    sorted_transaction_list = sorted(transaction_list, key=lambda d: d['timestamp'])

    # while loop. Breaks when the position reaches the end of the list.
    while position_transaction_list < len(transaction_list):

        # Get a dictionary from the list
        dictionary_element = sorted_transaction_list[position_transaction_list]
        
        # Get the current points in the pull dictionary
        points = dictionary_element['points']

        # If we still have points proceed intot the if statement.
        if count_switch == True:
            # If the points are less then the amount spent, return zero to the current dictionary  
            #  balanace. Otherwise, subtract the remainding points and return the difference. 
            if points <= points2subtract:
                points2subtract = points2subtract - points
                dictionary_element['points'] = 0
                cost2payer_list.append({'payer': dictionary_element['payer'], 'points':-points})
            else:
                points = points - points2subtract
                dictionary_element['points'] = points
                count_switch = False
                cost2payer_list.append({'payer': dictionary_element['payer'], 'points':-points2subtract})
        
        # Position Tracker
        position_transaction_list = position_transaction_list + 1

        # Creates a list of all unique payers
        if dictionary_element['payer'] not in payer_list:
            payer_list.append(dictionary_element['payer'])


    # Iterate throught  the unique payer list.
    for payer in payer_list:

        # create a list of all dictionaries with a specific payer
        payer_history = [element for element in sorted_transaction_list if element['payer'] == payer]

        payer_history_Cost =  [element for element in cost2payer_list if element['payer'] == payer]

        # Add the payer and the current sum balance to the report field
        balance_report[payer] = sum(item['points'] for item in payer_history)

        cost_report[payer] = sum(item['points'] for item in payer_history_Cost)


    # Switch to determine the report requested.
    if point_balance == True:
        return balance_report
    else:
        return cost_report
    

In [50]:
transaction_list = [{"payer": "DANNON", "points": 1000, "timestamp": "2020-11-02T14:00:00Z" }
,{"payer": "UNILEVER", "points": 200, "timestamp": "2020-10-31T11:00:00Z" }
,{"payer": "DANNON", "points": -200, "timestamp": "2020-10-31T15:00:00Z" }
,{"payer": "MILLER COORS","points": 10000,"timestamp": "2020-11-01T14:00:00Z" }
,{"payer": "DANNON", "points": 300, "timestamp": "2020-10-31T10:00:00Z" }]

#Then you call your spend points route with the following request: 

spent_points = { "points": 5000 }


transaction_return(transaction_list, spent_points, point_balance =False)

{'DANNON': -100, 'UNILEVER': -200, 'MILLER COORS': -4700}